In [ ]:
# Recreate it to srape Asda.com/  Morrisons.com / Sainsburys.com  / Ocado.com

# https://youtu.be/XQgXKtPSzUI
    
# Asp.net page that uses java script to validate user input string 
# (for validation text with numbers, 6 characters and on space)

In [ ]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
# For chrome and firefox
driver = webdriver.Firefox()
#driver = webdriver.Chrome()


In [ ]:
url = 'https://www.amazon.com'
driver.get(url)

In [ ]:
'https://www.amazon.com/s?k='
'ultrawide+monitor&crid=267UBIUB9WG7N&sprefix=ultrawide+%2Caps%2C397&ref=nb_sb_ss_ts-a-p_1_10'

In [ ]:
'https://www.amazon.com/s?k='
'iphone+12+pro+max&crid=36HYKGS762RC5&sprefix=ipho%2Caps%2C409&ref=nb_sb_noss_2'

In [ ]:
'https://www.amazon.com/s?k=ultrawide+monitor&crid=1CJT5GPM3T41H&sprefix=ultra%2Caps%2C395&ref=nb_sb_ss_ts-a-p_1_5'

In [ ]:
'https://www.amazon.com/s?k='
'playstation+5+console&crid=3QTMXQ2UAA1CO&sprefix=plays%2Caps%2C400&ref=nb_sb_ss_ts-a-p_9_5'

In [ ]:
def get_url(search_term):
    """ Generate a url from search term"""
    template = 'https://www.amazon.com/s?k=' + \
    '{}&crid=267UBIUB9WG7N&sprefix=ultrawide+%2Caps%2C397&ref=nb_sb_ss_ts-a-p_1_10'
    search_term = search_term.replace(' ','+')
    return template.format(search_term)

In [ ]:
url = get_url('ultrawide monitor')

In [ ]:
print(url)

In [ ]:
driver.get(url)

## Exract the collection

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
results = soup.find_all('div',{'data-component-type': 's-search-result'} )

In [ ]:
len(results)

## Prototype the record

In [ ]:
item = results[0]

In [ ]:
atag = item.h2.a

In [ ]:
description = atag.text.strip()

In [ ]:
description

In [ ]:
url = 'https://www.amazon.com' + atag.get('href')

In [ ]:
price_parent = item.find('span','a-price')

In [ ]:
price = price_parent.find('span','a-offscreen').text

In [ ]:
rating = item.i.text

In [ ]:
review_count = item.find('span',{'class': 'a-size-base','dir': 'auto'}).text

## Generalise the pattern

In [ ]:
def extract_record(item):
    """ Extract and return data for a single record """
    # description and url 
    try:
        atag = item.h2.a
        description = atag.text.strip()
        url = 'https://www.amazon.com' + atag.get('href')
    except AttributeError:
        description = ''
        
    # price
    try:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    # rank and rating
    try:
        rating = item.i.text
        review_count = item.find('span',{'class': 'a-size-base','dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result
    
    

In [ ]:
records = []
results = soup.find_all('div',{'data-component-type': 's-search-result'} )

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [ ]:
records[0]

In [ ]:
for row in records:
    print(row[1])

## Getting the next page

In [ ]:
def get_url(search_term):
    """ Generate a url from search term"""
    template = 'https://www.amazon.com/s?k=' + \
    '{}&crid=267UBIUB9WG7N&sprefix=ultrawide+%2Caps%2C397&ref=nb_sb_ss_ts-a-p_1_10'
    search_term = search_term.replace(' ','+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&page={}'
    
    return url

In [ ]:
url = get_url('ultrawide monitor')

In [ ]:
driver.get(url.format(4))

## Putting it all together

In [ ]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver



def get_url(search_term):
    """ Generate a url from search term"""
    template = 'https://www.amazon.com/s?k=' + \
    '{}&crid=267UBIUB9WG7N&sprefix=ultrawide+%2Caps%2C397&ref=nb_sb_ss_ts-a-p_1_10'
    search_term = search_term.replace(' ','+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&page{}'
    
    return url

def extract_record(item):
    """ Extract and return data for a single record """
    # description and url 
    try:
        atag = item.h2.a
        description = atag.text.strip()
        url = 'https://www.amazon.com' + atag.get('href')
    except AttributeError:
        description = ''
        
    # price
    try:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    # rank and rating
    try:
        rating = item.i.text
        review_count = item.find('span',{'class': 'a-size-base','dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result
    
def main(search_term):
    """ Run main program routine """
    # For chrome and firefox
    driver = webdriver.Firefox()
    #driver = webdriver.Chrome()
    
    records = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'} )

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    # save data to csv file
    with open('results.csv','w',newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Review_count', 'Url'])
        writer.writerows(records)

In [ ]:
main('ultrawide monitor')

## SCRAPING MORRISONS.COM

In [ ]:
url_morrisons = 'https://groceries.morrisons.com/on-offer'

In [ ]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
# For chrome and firefox
driver = webdriver.Firefox()
#driver = webdriver.Chrome()


In [ ]:
try:
    driver.get(url_morrisons)
except:
    print('Page not found')

In [ ]:
search_input = driver.find_element_by_xpath('//input[@id="search"]')

In [ ]:
search_input.send_keys('butter')

In [ ]:
search_input.send_keys(Keys.RETURN)

## Exract the collection

In [ ]:
results = driver.find_elements_by_xpath('.//li[@class="fops-item fops-item--cluster"]')

In [ ]:
results[218:221]

## Prototype the record

In [ ]:
item = results[21]

In [ ]:
item

In [ ]:
product_name = item.find_element_by_xpath('.//div[@class="fop-content"]//h4').get_attribute("title")#h4.span.text
product_name

In [ ]:
quantity = item.find_element_by_xpath('.//div[@class="fop-content"]//span[@class="fop-catch-weight"]').text
quantity

In [ ]:
price = item.find_element_by_xpath('.//div[@class="price-group-wrapper"]//span[@class="fop-price"]').text
price

In [ ]:
rating = item.find_element_by_xpath('.//div[@class="review-wrapper"]/span/span[@class="fop-rating-inner"]').get_attribute("title")
rating

In [ ]:
atag = item.find_element_by_xpath('.//div[@class="fop-contentWrapper"]/a').get_attribute("href")
atag
#('div',{'class','fop-contentWrapper'}).a

In [ ]:
product_url = atag #.attrib['href']
product_url 

## Generalise the pattern

In [ ]:
from selenium.common.exceptions import NoSuchElementException

In [ ]:
def extract_morrisons_record(item):
    """ Extract and return data for a single record """
    # description and url 
    try:
        price =''
        product_name = item.find_element_by_xpath('.//div[@class="fop-content"]//h4').get_attribute("title")
        price = item.find_element_by_xpath('.//div[@class="price-group-wrapper"]//span[@class="fop-price"]').text
        product_url = item.find_element_by_xpath('.//div[@class="fop-contentWrapper"]/a').get_attribute("href")
       
    except AttributeError:
        product_name = 'NotAvailable'
        price = 'NotAvailable'
        product_url = 'NotAvailable'
    except NoSuchElementException:
        product_url = 'wrong tag'
    
    # price
    try:
        quantity = item.find_element_by_xpath('.//div[@class="fop-content"]//span[@class="fop-catch-weight"]').text
        rating = rating = item.find_element_by_xpath('//div[@class="review-wrapper"]/span/span[@class="fop-rating-inner"]').get_attribute("title")
    except AttributeError:
        rating = 'NotAvailable'
        quantity = 'NotAvailable'
    
    result = (product_name, quantity, price,product_url, rating)
    
    return result
    
    

In [ ]:
# For chrome and firefox
driver = webdriver.Firefox()
#driver = webdriver.Chrome()
try:
    driver.get(url_morrisons)
except:
    print('Page not found')
    


In [ ]:
search_input = driver.find_element_by_xpath('//input[@id="search"]')
search_input.send_keys('onions')
search_input.send_keys(Keys.RETURN)

In [ ]:

records = []
results = driver.find_elements_by_xpath('//div[@class="fop-contentWrapper"]')

for item in results:
    record = extract_morrisons_record(item)
    if record:
        records.append(record)

In [ ]:
len(records)

In [ ]:
for row in records:
    print(row)

In [ ]:

# save data to csv file
save_to_csv('onions')

In [ ]:
def save_to_csv(filename):
    with open(filename + '.csv','w',newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Product Name', 'Quantity','Price', 'Url','Rating'])
        writer.writerows(records)

## SCRAPING TWITTER

In [32]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Firefox

In [40]:
# create web driver for firefox
driver = Firefox()

In [41]:
driver.get('https://www.twitter.com/login')

In [42]:
username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('0703960254')

In [43]:
my_password = getpass()

········


In [44]:
password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)

In [45]:
password.send_keys(Keys.RETURN)

In [46]:
search_input_twitter = driver.find_element_by_xpath('//input[@data-testid="SearchBox_Search_Input"]')
search_input_twitter.send_keys('#NBSFrontline')

In [47]:
search_input_twitter.send_keys(Keys.RETURN)

In [49]:
driver.find_element_by_link_text('Latest').click()

In [50]:
cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')

In [51]:
len(cards)

8

In [52]:
card = cards[2]

In [53]:
username = card.find_element_by_xpath('./div[2]/div[1]//span').text

In [54]:
handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text

In [55]:
time = card.find_element_by_xpath('.//time').get_attribute('datetime')

In [56]:
# content of the tweet
comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
comment + ' ' + responding

'Replying to \n@nbstv\n and \n@NancyKalembe @NancyKalembe\n #NBSFrontline #NBSUpdates , to solve the issue of an unleveled playing ground regarding campaign funding, a certain maximum budget by candidates should be enforced by law e.g. 200million. Whoever spends more than stipulated would be disqualified by EC.'

In [57]:
reply_count = card.find_element_by_xpath('.//div[@data-testid="reply"]').text

In [58]:
retweet_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text

In [59]:
like_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text

In [60]:
def get_tweet_data(card):
    """ Extract data from tweet data """
    username = card.find_element_by_xpath('./div[2]/div[1]//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    # content of the tweet
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_count = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    
    tweet = (username, handle, postdate, text, reply_count, retweet_count, like_count)
    return tweet

In [61]:
get_tweet_data(card)

('Beacon',
 '@BeaconUg',
 '2020-12-10T21:14:12.000Z',
 'Replying to \n@nbstv\n and \n@NancyKalembe@NancyKalembe\n #NBSFrontline #NBSUpdates , to solve the issue of an unleveled playing ground regarding campaign funding, a certain maximum budget by candidates should be enforced by law e.g. 200million. Whoever spends more than stipulated would be disqualified by EC.',
 '',
 '',
 '')

In [62]:
tweet_data = []
for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)

In [63]:
tweet_data[4]

('I Hate UMEME',
 '@Umwamikazi11',
 '2020-12-10T21:03:29.000Z',
 'As If Ssegona has been on a media tour today. Saw him on BBS terefayina in the morning. Now this #nbsfrontline',
 '2',
 '',
 '')

In [64]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

## Putting it all together

In [104]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Firefox

def get_tweet_data(card):
    """ Extract data from tweet data """
#     ignored_exceptions=(StaleElementReferenceException,)
#     sleep(5)
    try:
        
        username = card.find_element_by_xpath('.//span').text
    #WebDriverWait(driver, 20,ignored_exceptions=ignored_exceptions).until(EC.element_to_be_clickable((By.XPATH, \
    #'./div[2]/div[1]//span')))
    except StaleElementReferenceException:
        print('Stale element for username')
        username = ''
        
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except StaleElementReferenceException:
        handle = ''
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    except StaleElementReferenceException:
        postdate = ''
    
    try:
        
        # content of the tweet
        comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
        responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    
        text = comment + responding
    except StaleElementReferenceException:
        text = ''
    try:
        reply_count = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    except StaleElementReferenceException:
        reply_count = ''
        
    try:
        retweet_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    except StaleElementReferenceException:
        retweet_count = ''
    try:
        like_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    except StaleElementReferenceException:
        like_count = ''
        
    
    tweet = (username, handle, postdate, text, reply_count, retweet_count, like_count)
    return tweet

# create web driver for firefox
driver = Firefox()
driver.get('https://www.twitter.com/login')
driver.maximize_window()

username = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, \
    '//input[@name="session[username_or_email]"]')))
        

#username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('0703960254')
my_password = getpass()

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(1)

search_input_twitter = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, \
    '//input[@data-testid="SearchBox_Search_Input"]')))
#search_input_twitter = driver.find_element_by_xpath('//input[@data-testid="SearchBox_Search_Input"]')
search_input_twitter.send_keys('#venturecapital')
search_input_twitter.send_keys(Keys.RETURN)

# navigate to the latest tab
# try:
element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT, "Latest"))) 
    #'//input[@data-testid="SearchBox_Search_Input"]')))
element.click()
# except:
#     print('Could not access the "Latest" tab. Perhaps you are using a different language')
    
ignored_exceptions=(StaleElementReferenceException,)
    
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        current_position = driver.execute_script("return window.pageYOffset;")
        if last_position == current_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt to scroll again
        else:
            last_position = current_position
            break

········
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username
Stale element for username


In [105]:
len(data)

375

In [107]:
data[374]

('WWW.WIX.VC Venture Capital',
 '@Nauticus6',
 '2020-12-09T08:47:11.000Z',
 'Replying to \n@RampCapitalLLCEasy you buy a premium VC domain name and wait.  #vc #VentureCapital #virtualcompany #virtualclass #virtualcurrency #virtualcoin #domainsforsale',
 '',
 '',
 '')

In [92]:
def save_twitter_to_csv(filename):
    with open(filename + '.csv','w',newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Username', 'Handle','Postdate', 'Text','Reply Count','Retweet Count','Like Count'])
        writer.writerows(data)

In [108]:
save_twitter_to_csv('venturecapital')

In [90]:

# all_Links = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='pages-2']/div/ul/li/a[@href]")))

# for link in all_Links:
#     print link.get_attribute("href")
#     links.append(link.get_attribute("href"))

# for link in links:
#     driver.get(link)

In [ ]:
# links = driver.find_elements_by_xpath('//*[@id="pages-2"]/div/ul/li/a')
# links_hrefs = [link.get_attribute('href') for link in links]

# for i in links_hrefs:
#     driver.get(i)
